In [1]:
#!pip install grequests

In [2]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="grequests")
os.environ['GEVENT_SUPPORT'] = 'True'
#import grequests
import requests
from bs4 import BeautifulSoup
import json
import concurrent.futures
import urllib3
import time

In [3]:
####### INUTIL #############
#pd.read_csv("annotators.csv")

In [4]:
#pd.read_excel("annotations.xlsx")

In [5]:
df = pd.read_excel("labeled_dataset.xlsx").drop(["Unnamed: 0"], axis = 1)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
df.dropna(subset=['news_link'], inplace=True)

In [7]:
df.columns

Index(['sentence', 'news_link', 'outlet', 'topic', 'type', 'group_id',
       'num_sent', 'Label_bias', 'Label_opinion', 'article', 'biased_words4'],
      dtype='object')

In [8]:
df.shape

(1681, 11)

In [9]:
df['type'].unique()

array(['center', 'left', 'right'], dtype=object)

In [10]:
df['topic'].unique()

array(['elections-2020', 'sport', 'immigration', 'environment',
       'abortion', 'student-debt', 'vaccines', 'white-nationalism',
       'coronavirus', 'trump-presidency', 'middle-class', 'gender',
       'international-politics-and-world-news', 'gun-control'],
      dtype=object)

In [11]:
def get_news_batch(urls):
    results = []
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(get_news, i, url) for i, url in enumerate(urls)]
        
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            results.append(result)
    
    return dict(results)

In [12]:
def get_news(index, url):
    try:
        http = urllib3.PoolManager()
        response = http.request("GET", url)
    except:
        #time.sleep(1)
        return (index, ("ERRO - 1"))
        #return get_news(index, url)

    if response.status != 200:
      # Trata a resposta de erro
      if response.status == 429:
        time.sleep(3)
        return get_news(index, url)
      
      soup = BeautifulSoup(response.read(), "html.parser")
      error_message = soup.get_text()
      return (index, (f"ERRO - request: {response.status}"))
    
    soup = BeautifulSoup(response.read(), "html.parser")
    paragrafos = soup.find_all("p")
    
    texto_paragrafos = []
    
    try:
        for paragrafo in paragrafos:
            texto_paragrafos.append(paragrafo.get_text())
    except:
        texto_paragrafos.append("ERRO - 2")

    if len(texto_paragrafos) == 0 :
      soup = BeautifulSoup(response.data, "html.parser")
      paragrafos = soup.find_all("p")
      
      texto_paragrafos = []
      
      try:
          for paragrafo in paragrafos:
              texto_paragrafos.append(paragrafo.get_text())
      except:
          texto_paragrafos.append("ERRO - 3")

    
    return (index, tuple(texto_paragrafos))


In [13]:
urls = df["news_link"].tolist()
dict_news_text = get_news_batch(urls)

In [14]:
len(urls)

1681

In [15]:
len(dict_news_text.keys())

1681

In [16]:
dict_news_series = pd.Series(dict_news_text)

In [17]:
df['full_article'] = dict_news_series

Problematicos:

In [18]:
#zerados = df.loc[df['full_article'].apply(len) == 0]


In [19]:
errados_1 = df.loc[df['full_article'].apply(lambda x: 'ERRO - 1' in str(x))]

In [20]:
errados_1.shape

(7, 12)

In [21]:
errados_1

,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4,full_article
922,An avowed white supremacist pleaded guilty on ...,https://eu.usatoday.com/story/news/nation/2020...,usa-today,white-nationalism,center,47,1,No agreement,Entirely factual,NaN,[],ERRO - 1
1002,"Malinowski, a Democrat representing New Jersey...",https://www.breitbart.com/politics/2020/04/09/...,breitbart,coronavirus,right,51,1,Non-biased,Entirely factual,Rep. Tom Malinowski (D-NJ) reportedly said on ...,[],ERRO - 1
1058,Part of the reason billionaires can swoop in a...,https://www.nbcnews.com/think/opinion/jeff-bez...,msnbc,middle-class,left,53,1,Biased,Expresses writer’s opinion,"On Feb. 17, Amazon CEO and occasional richest-...","['swoop', 'saviors', 'failed']",ERRO - 1
1223,Background checks required to buy firearms hav...,https://eu.usatoday.com/story/money/2020/04/02...,usa-today,gun-control,center,62,1,Non-biased,Somewhat factual but also opinionated,NaN,[],ERRO - 1
1484,"In a bizarre and unexpected twist, pop super s...",https://www.breitbart.com/entertainment/2020/0...,breitbart,gender,right,75,1,Biased,Somewhat factual but also opinionated,"In a bizarre and unexpected twist, pop super s...","['smeared', 'bizarre', 'unexpected', 'twist', ...",ERRO - 1
1517,"John William King, 44, a white supremacist who...",https://eu.usatoday.com/story/news/nation/2019...,usa-today,white-nationalism,center,76,1,Non-biased,Somewhat factual but also opinionated,Texas executes white supremacist convicted in ...,['gruesome'],ERRO - 1
1567,The Republicans’ corporate tax cuts also magni...,https://www.nbcnews.com/business/economy/trump...,msnbc,middle-class,left,79,1,Biased,Expresses writer’s opinion,Get breaking news alerts and special reports. ...,"['economic', 'inequality', 'magnified']",ERRO - 1


In [34]:
errados_2 = df.loc[df['full_article'].apply(lambda x: 'ERRO - 2' in str(x))]

In [35]:
errados_2.shape

(0, 12)

In [36]:
errados_request = df.loc[df['full_article'].apply(lambda x: 'ERRO - request' in str(x))]

In [37]:
errados_request.shape

(241, 12)

In [26]:
errados_request[['news_link', 'full_article']]

,news_link,full_article
4,https://thefederalist.com/2020/03/11/woman-who...,ERRO - request: 403
25,https://thefederalist.com/2020/05/28/hannah-ga...,ERRO - request: 404
27,https://www.reuters.com/article/us-mississippi...,ERRO - request: 403
37,https://www.breitbart.com/politics/2017/04/12/...,ERRO - request: 404
39,https://www.reuters.com/article/us-davos-meeti...,ERRO - request: 403
...,...,...
1658,https://www.reuters.com/article/us-usa-white-n...,ERRO - request: 403
1660,https://www.nbcnews.com/politics/2020-election...,ERRO - request: 403
1667,https://eu.usatoday.com/story/news/politics/20...,ERRO - request: 403
1668,https://www.usatoday.com/videos/news/nation/20...,ERRO - request: 403


In [27]:
df_exploded = errados_request.explode("full_article")
valores_unicos = df_exploded["full_article"].unique()
del df_exploded
valores_unicos

array(['ERRO - request: 403', 'ERRO - request: 404',
       'ERRO - request: 410'], dtype=object)

In [48]:
merged = df.merge(errados_request, how='left', indicator=True)
filtered_df = merged[merged['_merge'] == 'left_only'].drop(columns='_merge')

filtered_df = filtered_df.merge(errados_1, how= 'left', indicator = True)
filtered_df = filtered_df[filtered_df['_merge'] == 'left_only'].drop(columns='_merge')

In [62]:
filtered_df.dropna(subset = ['full_article'], inplace = True)

In [67]:
filtered_df.to_csv("final_dataset.csv", index = False)